In [64]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [65]:
df = pd.read_csv('..\\Datasets\\covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


**Numerical Columns**: age, fever <BR>

**LabelEncoder is used in has_covid**

**Nominal Categorical Columns:** gender, city <br>
Using ONE HOT ENCODER<br>
**Ordinal Categorical Columns:** cough<br>
Using ORDINAL ENCODER<br>
  

In [66]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [67]:
print(df['gender'].unique())
print(df['cough'].unique())
print(df['city'].unique())

['Male' 'Female']
['Mild' 'Strong']
['Kolkata' 'Delhi' 'Mumbai' 'Bangalore']


Preprocessing to apply:<br>
fever : simple imputer<br>
gender : one hot enocder<br>
cough : ordinal encoder<br>
city : one hot encoder<br>

In [68]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:5],df.iloc[:,-1] ,test_size=0.2,random_state=0)

In [69]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


# 1. The HARD WAY

Filling missing values in fever column using SIMPLE IMPUTER

In [70]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
# X_train_fever


Ordinal Encoding in Cough

In [71]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])
# X_train_cough

One hot Encoding for Gender, City

In [72]:
ohe = OneHotEncoder(drop = 'first',sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.transform(X_train[['gender','city']])
print(type(X_test_gender_city))

<class 'numpy.ndarray'>


Extracting Age

In [80]:
X_train_age = X_train.drop(columns=['gender','city','cough','fever']).values
# X_train_age
X_test_age = X_test.drop(columns=['gender','city','cough','fever']).values
X_train_age.shape

(80, 1)

In [81]:
X_train_transformed = np.concatenate((X_train_age,X_train_gender_city,X_train_cough,X_train_fever), axis = 1)
X_train_transformed = np.concatenate((X_train_age,X_train_gender_city,X_train_cough,X_train_fever), axis = 1)

In [83]:
X_train_transformed.shape

(80, 7)

# 2. The Column Transformer Way

In [88]:
from sklearn.compose import ColumnTransformer
trans = ColumnTransformer(transformers=[
                            ('tnf1', SimpleImputer(), ['fever']),
                            ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
                            ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender','city'])
                            ], remainder='passthrough')

In [90]:
trans.fit_transform(X_train).shape

(80, 7)

In [91]:
trans.fit_transform(X_test).shape

(20, 7)